In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torchvision import models, datasets, transforms
import torch.optim as optim
from torch import nn
import os
from PIL import Image
import numpy as np

In [ ]:
!nvidia-smi
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
# 모델 불러오기 가장 유명한 resnet18을 이용하여 사용 (기존에 학습된것을 이용 기존 학습의 경우 1000개의 class를 구분 짓는 모델이여서 마지막 fully connected layer에서 1000을 현재 데이터에 맞는 3으로 변경)
import torch.nn as nn
import torch.nn.functional as F
import torchvision
#net = torchvision.models.vgg16(pretrained = True)
#net.classifier[6] = nn.Linear(4096,3)
net = torchvision.models.resnet18(pretrained = True)
#net = torchvision.models.resnet34(pretrained = True)
#net = torchvision.models.resnet50(pretrained = True)
net.fc = nn.Linear(net.fc.in_features,3)
net = net.to(0)

# loss, optimizer 설계
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.0001)
optimizer = optim.SGD(net.parameters(), lr=0.0001)

In [ ]:
classes = ('Covid', 'Normal', 'Viral Pneumonia')

In [ ]:
##########################################

for epoch in range(5):  # 얼마나 반복시키면서 학습할껀지
    trainset = datasets.ImageFolder(root='/content/drive/MyDrive/Covid19-dataset/train/',transform = transforms.Compose([transforms.Grayscale(3),transforms.Resize((512,512)),transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),transforms.RandomRotation(5),transforms.ToTensor()]))
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                              shuffle=True,num_workers = 4)

    testset = datasets.ImageFolder(root='/content/drive/MyDrive/Covid19-dataset/test/',transform = transforms.Compose([transforms.Grayscale(3),transforms.Resize((512,512)),transforms.ToTensor()]))
    testloader = torch.utils.data.DataLoader(testset, batch_size=16,
                                              shuffle=False,num_workers = 4)
    running_loss = 0.0
    temp,total = 0, 0
    for i, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to(0), labels.to(0)
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        temp += (predicted == labels).sum().item()
        total += inputs.shape[0]

        # print statistics
        running_loss += loss.item()
        if i % 2 == 0:    # 매번 출력하면 힘드니까 2번당 1번씩 출력
            print('[%d, %5d] trainloss: %.3f, trainaccuracy : %.2f' %
                  (epoch + 1, i + 1, running_loss / 2, (temp/total)))
            #print('train accuracy : %.2f' % (temp / total))
            running_loss = 0.0
            temp,total = 0, 0

    for i, data in enumerate(testloader):
        inputs, labels = data
        inputs, labels = inputs.to(0), labels.to(0)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        _, predicted = torch.max(outputs.data, 1)
        temp += (predicted == labels).sum().item()
        total += inputs.shape[0]

        # print statistics
        running_loss += loss.item()
        if i % 2 == 0:    # 매번 출력하면 힘드니까 2번당 1번씩 출력
            print('[%d, %5d] testloss: %.3f, testaccuracy : %.2f' %
                  (epoch + 1, i + 1, running_loss / 2, (temp/total)))
            running_loss = 0.0
            temp,total = 0, 0
print('Finished Training')

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(0), labels.to(0)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('test셋 결과물: %d %%' % (
    100 * correct / total))

In [ ]:
############# 최종 평가
mat = torch.zeros((3,3))
class_correct = list(0. for i in range(3))
class_total = list(0. for i in range(3))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(0), labels.to(0)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted, labels)
        c = (predicted == labels).squeeze()
        for i in range(2):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
        for i,x in enumerate(labels):
          mat[predicted[i],x] += 1
print(mat)
for i in range(3):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))